In [12]:
import os.path
from pathlib import Path
import pandas as pd

path = '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation'

image_dir = Path(path)

filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.PNG')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

['/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation/yellow_mosaic/pYM (70)78_saved753027.jpg'
 '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation/yellow_mosaic/YM (86)95_saved1033734.jpg'
 '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation/yellow_mosaic/YM (35)39_saved651211.jpg'
 '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation/yellow_mosaic/pYM (34)38_saved1061110.jpg'
 '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation/yellow_mosaic/YM (28)31_saved10498.jpg'
 '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation/yellow_mosaic/pYM (57)63_saved1012522.jpg'
 '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/augumentation/yellow_mosaic/pYM (50)56_saved1002119.jpg'
 '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classif

In [35]:
import os
import cv2
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from skimage.feature import canny
from skimage.exposure import rescale_intensity
import numpy as np

def save_feature_image(image, op_path, method, file_suffix=''):
    method_path = os.path.join(op_path, method)
    if not os.path.exists(method_path):
        os.makedirs(method_path)
    save_path = os.path.join(method_path, f"{file_suffix}.jpg")
    cv2.imwrite(save_path, image)

def lbp_feature(image, op_path, filename):
    lbp_image = local_binary_pattern(image, P=8, R=1, method='uniform')
    save_feature_image(lbp_image, op_path, 'lbp', filename)

def canny_feature(image, op_path, filename):
    canny_img = canny(image, sigma=1.0)
    canny_image = (canny_img.astype(np.uint8) * 255).astype(np.uint8)
    save_feature_image(canny_image, op_path, 'canny', filename)

def hog_feature(image, op_path, filename):
    features, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)
    hog_image_rescaled = rescale_intensity(hog_image, in_range=(0, 10))
    save_feature_image(hog_image_rescaled, op_path, 'hog', filename)

def log_feature(image, op_path, filename):
    log_image = cv2.Laplacian(image, cv2.CV_16S, ksize=3)
    save_feature_image(log_image, op_path, 'log', filename)

op_path_base = '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/feature_extraction/'

for row in image_df.itertuples(index=False):
    original_file = cv2.imread(row.Filepath)
    img = cv2.cvtColor(original_file, cv2.COLOR_BGR2GRAY)
    filename = os.path.basename(row.Filepath)
    op_path = os.path.join(op_path_base, row.Label)
    lbp_feature(img, op_path, filename)
    log_feature(img, op_path, filename)
    canny_feature(img, op_path, filename)
    hog_feature(img, op_path, filename)


In [51]:
import os.path
from pathlib import Path
import pandas as pd

path = '/Users/kacperskurski/DataspellProjects/Soybean_Disease_Leaf_Classification/feature_extraction/'

image_dir = Path(path)

filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.PNG')) + list(image_dir.glob(r'**/*.png'))

features = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
labels = list(map(lambda x: os.path.basename(os.path.dirname(os.path.dirname(x))), filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
features = pd.Series(features, name='Feature')

image_df_features = pd.concat([filepaths, labels, features], axis=1)

In [55]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import cv2
import numpy as np

X = np.array([cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY).flatten() for image_path in image_df_features.Filepath.values if cv2.imread(image_path) is not None])
y = np.array(image_df_features.Label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy}')


Model accuracy: 0.41968911917098445
